Banco de dados a ser usado

[Detecção de fraude em cartão de crédito](https://drive.google.com/file/d/1HAIgrwZospUvU5ybxvfc1SIqJ-Z2rMGh/view?usp=drive_link)


Fazer uma pipeline completa de otimização:

1. Separar em treino (70%), teste (20%) e validação (10%).
2. Fazer o tunning de hiperparametro para os modelos:

  a. Random Forest.
  * max_depth: uniforme de 5 a 10
  * n_estimators: uniforme de 100 a 500
  
  b. Xgboost
  * learning_rate: log uniforme de 0.01 a 0.3
  * max_depth: uniforme de 5 a 10
  * n_estimators: uniformde de 5 a 10
  
  c. SVM
  * C: uniforme de 1 a 1000
  * gamma: uniforme de .01 a 1
  
  d. Lasso
  * Alpha: 0.1 a 10
  
  e. Ridge
  * Alpha: 0.1 a 10

Todos executar 100 vezes.
3. Treinar cada um dos modelos com a melhor configuração
4. Definir o ponto de corte utilizando o banco de validação e como métrica o f1-score da classe minoritaria
5. Avaliar o classificador no banco de teste

In [1]:
from google.colab import files
uploaded = files.upload()

Saving creditcard_sample.csv to creditcard_sample.csv


In [6]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('creditcard_sample.csv')

In [20]:
data.Class.value_counts(normalize=True)

Class
0    0.909091
1    0.090909
Name: proportion, dtype: float64

In [4]:
X = data.drop(columns='Class')
Y = data.Class

In [7]:
np.mean(Y)

0.09090909090909091

In [8]:
import scipy
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

In [9]:
X_trainvalid, X_test, Y_trainvalid, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_train, X_valid, Y_train, Y_valid = train_test_split(X_trainvalid, Y_trainvalid, test_size=1/8, random_state=42)

print(np.mean(Y_valid))

0.08863636363636364


In [12]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
parameters = {'max_depth': scipy.stats.randint(5, 10),
              'n_estimators': scipy.stats.randint(100, 500)}

clf_rf = RandomizedSearchCV(model, parameters, scoring='f1_macro', n_iter=20, verbose=3)
clf_rf.fit(X_train, Y_train)
pd.DataFrame(clf_rf.cv_results_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5] END .....max_depth=6, n_estimators=497;, score=0.957 total time=   6.3s
[CV 2/5] END .....max_depth=6, n_estimators=497;, score=0.963 total time=   5.1s
[CV 3/5] END .....max_depth=6, n_estimators=497;, score=0.951 total time=   4.1s
[CV 4/5] END .....max_depth=6, n_estimators=497;, score=0.951 total time=   4.2s
[CV 5/5] END .....max_depth=6, n_estimators=497;, score=0.969 total time=   5.2s
[CV 1/5] END .....max_depth=7, n_estimators=414;, score=0.957 total time=   3.9s
[CV 2/5] END .....max_depth=7, n_estimators=414;, score=0.963 total time=   3.9s
[CV 3/5] END .....max_depth=7, n_estimators=414;, score=0.951 total time=   4.8s
[CV 4/5] END .....max_depth=7, n_estimators=414;, score=0.951 total time=   3.9s
[CV 5/5] END .....max_depth=7, n_estimators=414;, score=0.969 total time=   3.9s
[CV 1/5] END .....max_depth=8, n_estimators=447;, score=0.957 total time=   5.6s
[CV 2/5] END .....max_depth=8, n_estimators=447

AttributeError: module 'pandas' has no attribute 'Dataframe'

In [13]:
pd.DataFrame(clf_rf.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,4.939972,0.799732,0.051613,0.001176,6,497,"{'max_depth': 6, 'n_estimators': 497}",0.957244,0.962919,0.951467,0.951467,0.969034,0.958426,0.006797,2
1,4.043888,0.371587,0.045449,0.001804,7,414,"{'max_depth': 7, 'n_estimators': 414}",0.957244,0.962919,0.951467,0.951467,0.969034,0.958426,0.006797,2
2,4.905717,0.485275,0.054007,0.007299,8,447,"{'max_depth': 8, 'n_estimators': 447}",0.957244,0.962919,0.951467,0.951467,0.969034,0.958426,0.006797,2
3,1.871928,0.237639,0.024502,0.002666,7,180,"{'max_depth': 7, 'n_estimators': 180}",0.957244,0.962919,0.951467,0.951467,0.969034,0.958426,0.006797,2
4,1.927725,0.196645,0.024687,0.002001,7,187,"{'max_depth': 7, 'n_estimators': 187}",0.957244,0.962919,0.951467,0.951467,0.969034,0.958426,0.006797,2
5,4.065927,0.314742,0.048127,0.005951,7,398,"{'max_depth': 7, 'n_estimators': 398}",0.957244,0.962919,0.951467,0.951467,0.969034,0.958426,0.006797,2
6,2.223092,0.229667,0.024986,0.004420,9,183,"{'max_depth': 9, 'n_estimators': 183}",0.957244,0.962919,0.951467,0.957244,0.969034,0.959582,0.005954,1
7,2.676159,0.315097,0.036461,0.004336,5,337,"{'max_depth': 5, 'n_estimators': 337}",0.957244,0.962919,0.951467,0.945583,0.969034,0.957249,0.008253,20
8,3.178167,0.302260,0.036807,0.004940,8,292,"{'max_depth': 8, 'n_estimators': 292}",0.957244,0.962919,0.951467,0.946556,0.969034,0.957444,0.007983,17
9,3.098976,0.221009,0.041150,0.012452,8,286,"{'max_depth': 8, 'n_estimators': 286}",0.957244,0.962919,0.951467,0.951467,0.969034,0.958426,0.006797,2


In [45]:
from xgboost import XGBClassifier
model = XGBClassifier(objective='binary:logistic')
parameters = {'learning_rate' : scipy.stats.expon(scale=.1),
              'n_estimators': scipy.stats.randint(50, 500),
              'max_depth': scipy.stats.randint(5, 10)}


clf_xgb = RandomizedSearchCV(model, parameters, scoring='f1_macro', n_iter=20)
clf_xgb.fit(X_train, Y_train)
pd.DataFrame(clf_xgb.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,2.867044,1.471283,0.020964,0.012694,0.040488,9,419,"{'learning_rate': 0.04048813334282804, 'max_de...",0.957244,0.957992,0.946556,0.938068,0.959404,0.951853,0.008269,18
1,2.015872,1.071476,0.022313,0.008367,0.06931,8,120,"{'learning_rate': 0.06931046686398694, 'max_de...",0.957244,0.957992,0.946556,0.947490,0.969553,0.955767,0.008373,3
2,0.830063,0.284220,0.032084,0.014996,0.195297,9,60,"{'learning_rate': 0.19529650314147198, 'max_de...",0.957244,0.957992,0.946556,0.947490,0.964179,0.954692,0.006715,5
3,4.994393,1.262738,0.021154,0.004288,0.018766,9,239,"{'learning_rate': 0.018765576704936254, 'max_d...",0.957244,0.962919,0.951467,0.947490,0.964179,0.956660,0.006436,2
4,4.574014,1.662510,0.028251,0.012702,0.036425,8,470,"{'learning_rate': 0.03642504414613174, 'max_de...",0.957244,0.957992,0.946556,0.942739,0.959404,0.952787,0.006790,15
5,3.076387,1.125455,0.029938,0.010354,0.090293,7,467,"{'learning_rate': 0.09029267624209326, 'max_de...",0.957244,0.957992,0.946556,0.947490,0.964773,0.954811,0.006885,4
6,2.343572,1.673170,0.023026,0.012961,0.184952,8,494,"{'learning_rate': 0.18495179322217045, 'max_de...",0.957244,0.957992,0.941730,0.938068,0.964773,0.951962,0.010258,16
7,0.484108,0.018788,0.011465,0.000649,0.167864,7,388,"{'learning_rate': 0.16786391927974403, 'max_de...",0.957244,0.957992,0.946556,0.942739,0.964773,0.953861,0.008058,8
8,0.562655,0.019219,0.011539,0.001092,0.103955,9,324,"{'learning_rate': 0.10395478812600961, 'max_de...",0.957244,0.957992,0.946556,0.938068,0.964773,0.952927,0.009445,13
9,0.478736,0.025519,0.011134,0.000595,0.100576,5,266,"{'learning_rate': 0.10057598908593486, 'max_de...",0.957244,0.963561,0.946556,0.952325,0.969553,0.957848,0.008100,1


In [15]:
from sklearn.svm import SVC
model = SVC()
parameters = {'C': scipy.stats.randint(1, 1000),
              'gamma': scipy.stats.uniform(.01, 1)}

clf_svm = RandomizedSearchCV(model, parameters, scoring='f1_macro', n_iter=20)
clf_svm.fit(X_train, Y_train)
pd.DataFrame(clf_svm.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.505663,0.011670,0.129761,0.006597,185,0.289976,"{'C': 185, 'gamma': 0.2899756473291206}",0.529578,0.529578,0.562002,0.512535,0.511484,0.529035,0.018263,2
1,0.579377,0.150751,0.176621,0.059881,738,0.126281,"{'C': 738, 'gamma': 0.12628110371183365}",0.529578,0.529578,0.562002,0.512535,0.511484,0.529035,0.018263,2
2,0.755084,0.340863,0.160069,0.051722,542,0.593533,"{'C': 542, 'gamma': 0.5935327062424495}",0.529578,0.529578,0.562002,0.512535,0.511484,0.529035,0.018263,2
3,0.505661,0.018841,0.133159,0.012523,405,0.505568,"{'C': 405, 'gamma': 0.5055677242232542}",0.529578,0.529578,0.562002,0.512535,0.511484,0.529035,0.018263,2
4,0.501755,0.012728,0.125193,0.003564,953,0.908917,"{'C': 953, 'gamma': 0.9089166911400359}",0.529578,0.529578,0.562002,0.512535,0.511484,0.529035,0.018263,2
5,0.671427,0.178690,0.209272,0.062778,168,0.807645,"{'C': 168, 'gamma': 0.8076448394416276}",0.529578,0.529578,0.562002,0.512535,0.511484,0.529035,0.018263,2
6,0.566651,0.137507,0.153178,0.050989,324,1.007799,"{'C': 324, 'gamma': 1.0077985974204204}",0.529578,0.529578,0.562002,0.512535,0.511484,0.529035,0.018263,2
7,0.515690,0.047845,0.129045,0.005256,648,0.993047,"{'C': 648, 'gamma': 0.9930466057686471}",0.529578,0.529578,0.562002,0.512535,0.511484,0.529035,0.018263,2
8,0.558855,0.081441,0.192392,0.125767,458,0.765673,"{'C': 458, 'gamma': 0.7656729001365747}",0.529578,0.529578,0.562002,0.512535,0.511484,0.529035,0.018263,2
9,0.748996,0.150892,0.226533,0.053441,959,0.693865,"{'C': 959, 'gamma': 0.6938648450922964}",0.529578,0.529578,0.562002,0.512535,0.511484,0.529035,0.018263,2


In [27]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='l1', solver='liblinear')
parameters = {'C': scipy.stats.uniform(0.1, 1)}

clf_lasso = RandomizedSearchCV(model, parameters, scoring='f1_macro', n_iter=20)
clf_lasso.fit(X_train, Y_train)
pd.DataFrame(clf_lasso.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.065388,0.018774,0.014799,0.003232,1.007175,{'C': 1.0071754383324247},0.941730,0.958712,0.952325,0.932328,0.964179,0.949855,0.011515,12
1,0.059915,0.027653,0.011790,0.004776,0.817189,{'C': 0.8171893403658085},0.941730,0.958712,0.952325,0.936989,0.964179,0.950787,0.010168,5
2,0.045336,0.014365,0.008520,0.003358,0.464243,{'C': 0.46424270984019256},0.936989,0.958712,0.952325,0.946556,0.964179,0.951752,0.009467,1
3,0.044355,0.015237,0.007891,0.003970,0.661248,{'C': 0.6612480596510337},0.936989,0.958712,0.952325,0.936989,0.964179,0.949838,0.011143,15
4,0.040954,0.007695,0.007145,0.001749,0.745035,{'C': 0.7450353412151239},0.936989,0.958712,0.952325,0.936989,0.964179,0.949838,0.011143,15
5,0.049669,0.019426,0.011414,0.005366,0.903586,{'C': 0.9035861748710778},0.936989,0.958712,0.952325,0.936989,0.964179,0.949838,0.011143,15
6,0.043173,0.016506,0.012074,0.004724,0.328414,{'C': 0.3284139056060893},0.932328,0.958712,0.952325,0.946556,0.964179,0.950820,0.010983,4
7,0.043121,0.016453,0.013061,0.007939,0.595661,{'C': 0.5956606943065166},0.936989,0.958712,0.952325,0.941730,0.964179,0.950787,0.010168,5
8,0.046078,0.022679,0.011650,0.006386,1.066553,{'C': 1.066553290201544},0.941730,0.958712,0.952325,0.932328,0.964179,0.949855,0.011515,12
9,0.033168,0.018128,0.011020,0.005638,0.68417,{'C': 0.6841702798473062},0.936989,0.958712,0.947490,0.936989,0.964179,0.948872,0.011095,20


In [28]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='l2', solver='liblinear')
parameters = {'C': scipy.stats.uniform(0.1, 1)}

clf_ridge = RandomizedSearchCV(model, parameters, scoring='f1_macro', n_iter=20)
clf_ridge.fit(X_train, Y_train)
pd.DataFrame(clf_ridge.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.048936,0.015907,0.008734,0.002336,0.461228,{'C': 0.4612284146369472},0.896347,0.951467,0.912933,0.894279,0.958712,0.922747,0.027284,1
1,0.048935,0.014971,0.008379,0.002456,0.117924,{'C': 0.11792386667659885},0.896347,0.951467,0.912933,0.894279,0.953150,0.921635,0.025872,14
2,0.043985,0.020249,0.008340,0.002482,0.649017,{'C': 0.6490171003034565},0.896347,0.951467,0.912933,0.894279,0.958712,0.922747,0.027284,1
3,0.044998,0.013622,0.007940,0.003653,0.594139,{'C': 0.5941390442294084},0.896347,0.951467,0.912933,0.894279,0.958712,0.922747,0.027284,1
4,0.055554,0.014369,0.012236,0.007439,0.973641,{'C': 0.9736413096086273},0.896347,0.951467,0.912933,0.894279,0.958712,0.922747,0.027284,1
5,0.038489,0.014109,0.005476,0.000332,0.269922,{'C': 0.26992215958673405},0.896347,0.951467,0.912933,0.894279,0.953150,0.921635,0.025872,14
6,0.034925,0.004703,0.005605,0.000678,1.04128,{'C': 1.041279754848968},0.896347,0.951467,0.912933,0.894279,0.958712,0.922747,0.027284,1
7,0.033754,0.003712,0.005855,0.000664,0.94997,{'C': 0.9499698574365867},0.896347,0.951467,0.912933,0.894279,0.958712,0.922747,0.027284,1
8,0.032609,0.002399,0.005800,0.000864,0.689839,{'C': 0.6898391583295478},0.896347,0.951467,0.912933,0.894279,0.958712,0.922747,0.027284,1
9,0.035442,0.007897,0.005373,0.000357,0.21338,{'C': 0.21337956710345876},0.896347,0.951467,0.912933,0.894279,0.953150,0.921635,0.025872,14


In [29]:
clf_rf.best_params_

{'max_depth': 9, 'n_estimators': 183}

In [30]:
clf_xgb.best_params_

{'learning_rate': 0.22068177430663485, 'n_estimators': 250}

In [31]:
clf_svm.best_params_

{'C': 450, 'gamma': 0.011045014122255439}

In [32]:
clf_lasso.best_params_

{'C': 0.46424270984019256}

In [33]:
clf_ridge.best_params_

{'C': 0.4612284146369472}

In [36]:
from sklearn.metrics import f1_score

def found_thr(predict, Y):
  best_thr = 0
  best_f1 = 0
  for thr in np.arange(0.01, 1, 0.001):
    predict_class = predict > thr
    f1 = f1_score(Y, predict_class)
    if f1 > best_f1:
      best_thr = thr
      best_f1 = f1
  return best_thr

In [37]:
model_rf = RandomForestClassifier(max_depth=clf_rf.best_params_['max_depth'],
                                  n_estimators=clf_rf.best_params_['n_estimators'])
model_rf.fit(X_train, Y_train)

thr_rf = found_thr(model_rf.predict_proba(X_valid)[:, 1], Y_valid)

print(thr_rf)

0.23599999999999982


In [47]:
model_xgb = XGBClassifier(learning_rate=clf_xgb.best_params_['learning_rate'],
                          n_estimators=clf_xgb.best_params_['n_estimators'],
                          max_depth=clf_xgb.best_params_['max_depth'])
model_xgb.fit(X_train, Y_train)

thr_xgb = found_thr(model_xgb.predict_proba(X_valid)[:, 1], Y_valid)

print(thr_xgb)

0.09099999999999993


In [41]:
model_svm = SVC(C=clf_svm.best_params_['C'], gamma=clf_svm.best_params_['gamma'])
model_svm.fit(X_train, Y_train)

SVC(C=450, gamma=0.011045014122255439)

In [54]:
model_lasso = LogisticRegression(C=clf_lasso.best_params_['C'],
                                 penalty='l1',
                                 solver='liblinear')
model_lasso.fit(X_train, Y_train)

thr_lasso = found_thr(model_lasso.predict_proba(X_valid)[:, 1], Y_valid)

print(thr_lasso)

0.1219999999999999


In [55]:
model_ridge = LogisticRegression(C=clf_ridge.best_params_['C'],
                                 penalty='l2',
                                 solver='liblinear')
model_ridge.fit(X_train, Y_train)

thr_ridge = found_thr(model_ridge.predict_proba(X_valid)[:, 1], Y_valid)

print(thr_ridge)

0.5139999999999996


In [51]:
from sklearn.metrics import classification_report

In [52]:
print(classification_report(Y_test, model_xgb.predict_proba(X_test)[:, 1] > thr_xgb))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       795
           1       0.88      0.89      0.89        85

    accuracy                           0.98       880
   macro avg       0.94      0.94      0.94       880
weighted avg       0.98      0.98      0.98       880



In [53]:
print(classification_report(Y_test, model_rf.predict_proba(X_test)[:, 1] > thr_rf))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       795
           1       0.95      0.86      0.90        85

    accuracy                           0.98       880
   macro avg       0.97      0.93      0.95       880
weighted avg       0.98      0.98      0.98       880



In [57]:
print(classification_report(Y_test, model_lasso.predict_proba(X_test)[:, 1] > thr_lasso))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       795
           1       0.87      0.88      0.88        85

    accuracy                           0.98       880
   macro avg       0.93      0.93      0.93       880
weighted avg       0.98      0.98      0.98       880



In [58]:
print(classification_report(Y_test, model_ridge.predict_proba(X_test)[:, 1] > thr_ridge))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97       795
           1       0.98      0.53      0.69        85

    accuracy                           0.95       880
   macro avg       0.97      0.76      0.83       880
weighted avg       0.95      0.95      0.95       880



In [59]:
print(classification_report(Y_test, model_svm.predict(X_test)))

              precision    recall  f1-score   support

           0       0.91      1.00      0.95       795
           1       0.67      0.02      0.05        85

    accuracy                           0.90       880
   macro avg       0.79      0.51      0.50       880
weighted avg       0.88      0.90      0.86       880

